<a href="https://colab.research.google.com/github/Amir-D-Shadow/Google-Colab/blob/main/Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numba import cuda,float64,int64
import numpy as np
import math

In [2]:
@cuda.jit("float64[:,:,:],float64[:,:,:],float64[:,:,:,:],int64,int64,int64,int64")
def func3D(A,dZ,W,stride,Hlim,Wlim,Clim):

  """
  A -- n_H_prev + 2 * opadH, n_W_prev + 2*opadW,n_C_prev
  dZ -- n_H,n_W,n_C
  W -- fH,fW,n_C_prev,n_C
  """

  n_H = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
  n_W = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
  n_C = cuda.threadIdx.z + cuda.blockDim.z * cuda.blockIdx.z

  if (n_H < Hlim) and (n_W < Wlim) and (n_C < Clim):

    fH,fW,n_C_prev,_ = W.shape

    for h in range(fH):

      for w in range(fW):

        for c in range(n_C_prev):

          IMG_H = n_H * stride + h
          IMG_W = n_W * stride + w

          A[IMG_H,IMG_W,c] = A[IMG_H,IMG_W,c] + W[h,w,c,n_C] * dZ[n_H,n_W,n_C]

In [3]:
opadH,opadW,stride = 2,2,1
m = 2
number_of_filters = 1

tmp = np.zeros((m,1,1,1)).astype(np.float64)
W = np.ones((2,2,1,number_of_filters)).astype(np.float64)
dZ = np.ones((m,2,2,number_of_filters)).astype(np.float64)
A = np.pad(tmp,((0,0),(opadH,opadH),(opadW,opadW),(0,0)),mode="constant",constant_values=(0,0))

m,n_H,n_W,n_C = dZ.shape 

threadsperblock = (2,2,2)

blockspergrid_H = int(math.ceil(n_H/threadsperblock[0]))
blockspergrid_W = int(math.ceil(n_W/threadsperblock[1]))
blockspergrid_C = int(math.ceil(n_C/threadsperblock[2]))

blockspergrid = (blockspergrid_H,blockspergrid_W,blockspergrid_C)
W_device = cuda.to_device(W)

for i in range(m):

  dZ_device = cuda.to_device(dZ[i,:,:,:].copy())
  A_device = cuda.to_device(A[i,:,:,:].copy())

  func3D[blockspergrid,threadsperblock](A_device,dZ_device,W_device,stride,n_H,n_W,n_C)
  cuda.synchronize()
  A[i,:,:,:] = A_device.copy_to_host()

res = A[:,opadH:-opadH,opadW:-opadW,:]

print(res)


[[[[1.]]]


 [[[1.]]]]


In [4]:
@cuda.jit
def test3D(A,B,out,Hlim,Wlim,Clim):

  x = cuda.threadIdx.x + cuda.blockDim.x*cuda.blockIdx.x
  y = cuda.threadIdx.y + cuda.blockDim.y*cuda.blockIdx.y
  z = cuda.threadIdx.z + cuda.blockDim.z*cuda.blockIdx.z

  if (x<Hlim) and (y<Wlim) and (z<Clim):

    out[x,y,z] = A[x,y,z]*B[x,y,z]

In [5]:
opadH,opadW,stride = 2,2,1
m = 2
number_of_filters = 1

A = np.full((3,3,3),6)
B = np.full((3,3,3),7)

n_H,n_W,n_C = A.shape

A_device = cuda.to_device(A)
B_device = cuda.to_device(B)
out_device = cuda.to_device(np.zeros((3,3,3)))

threadsperblock = (2,2,2)

blockspergrid_H = int(math.ceil(n_H/threadsperblock[0]))
blockspergrid_W = int(math.ceil(n_W/threadsperblock[1]))
blockspergrid_C = int(math.ceil(n_C/threadsperblock[2]))

blockspergrid = (blockspergrid_H,blockspergrid_W,blockspergrid_C)

test3D[blockspergrid,threadsperblock](A_device,B_device,out_device,n_H,n_W,n_C)
cuda.synchronize()
res = out_device.copy_to_host()
print(res)

[[[42. 42. 42.]
  [42. 42. 42.]
  [42. 42. 42.]]

 [[42. 42. 42.]
  [42. 42. 42.]
  [42. 42. 42.]]

 [[42. 42. 42.]
  [42. 42. 42.]
  [42. 42. 42.]]]


In [6]:
a = np.arange(4).reshape(2,2)
b = np.zeros((2,2))

c = np.array([a,b,b])
k1 = np.rot90(c,1,axes=(0,1))
k2 = np.rot90(c,1,axes=(1,2))
k3 = np.rot90(c,1,axes=(0,2))
print("c:",c,"\n")
print("k1:",k1,"\n")
print("k2:",k2,"\n")
print("k3",k3,"\n")


c: [[[0. 1.]
  [2. 3.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]] 

k1: [[[2. 3.]
  [0. 0.]
  [0. 0.]]

 [[0. 1.]
  [0. 0.]
  [0. 0.]]] 

k2: [[[1. 3.]
  [0. 2.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]] 

k3 [[[1. 0. 0.]
  [3. 0. 0.]]

 [[0. 0. 0.]
  [2. 0. 0.]]] 



In [7]:
opadH,opadW,stride = 2,2,1
m = 2
number_of_filters = 1

dA = np.full((3,3,3),6)
B = np.full((3,3,3),7)

n_H,n_W,n_C = A.shape

A_device = cuda.to_device(A)
B_device = cuda.to_device(B)
out_device = cuda.to_device(np.zeros((3,3,3)))

threadsperblock = (4,4,4)

blockspergrid_H = n_H
blockspergrid_W = n_W
blockspergrid_C = n_C

blockspergrid = (blockspergrid_H,blockspergrid_W,blockspergrid_C)

test3D[blockspergrid,threadsperblock](A_device,B_device,out_device,n_H,n_W,n_C)
cuda.synchronize()
res = out_device.copy_to_host()
print(res)

[[[42. 42. 42.]
  [42. 42. 42.]
  [42. 42. 42.]]

 [[42. 42. 42.]
  [42. 42. 42.]
  [42. 42. 42.]]

 [[42. 42. 42.]
  [42. 42. 42.]
  [42. 42. 42.]]]


In [8]:
@cuda.jit("float64[:,:,:,:],float64[:,:,:,:],float64[:,:,:,:],int64,int64,int64,int64")
def func3D_new(dA_prev,W,dZ,stride,Hlim,Wlim,Clim):
    
    """
    dA_prev -- (m,n_H_prev,n_W_prev,n_C_prev)
    W -- (fH,fW,n_C_prev,n_C)
    dZ -- (m,n_H,n_W,n_C)
    """
    
    IMG_H = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    IMG_W = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
    IMG_C = cuda.threadIdx.z + cuda.blockDim.z * cuda.blockIdx.z
    
    if IMG_H<Hlim and IMG_W<Wlim and IMG_C<Clim:
        
      fH,fW,n_C_prev,_ = W.shape
      m,n_H,n_W,n_C = dZ.shape
      for i in range(m):
        
        for h in range(fH):

          for w in range(fW):
            
            for c in range(n_C):

              dA_prev[i,IMG_H,IMG_W,IMG_C] = dA_prev[i,IMG_H,IMG_W,IMG_C] + W[h,w,IMG_C,c] * dZ[i,h,w,c]
                    

In [9]:
def zero_padding(img,padH,padW):

        """
        img : numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
        pad : amount of padding around each image on vertical and horizontal dimensions
        """

        return np.pad(img,((0,0),(padH,padH),(padW,padW),(0,0)),mode="constant",constant_values=(0,0))

def same_padding(img,stride,fH,fW):
    
        """
        img : numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
        fH -- Kernel Size (height)
        fW -- Kernel Size (width)
        s -- Stride
        """
        n_H, n_W = img.shape[1],img.shape[2]
        padH = int(math.ceil(((n_H-1)*stride+fH-n_H)/2))
        padW = int(math.ceil(((n_W-1)*stride+fW-n_W)/2))

        img_same_pad = zero_padding(img,padH,padW)

        return img_same_pad,padH,padW

In [10]:
opadH,opadW,stride = 1,1,1
m = 1
number_of_filters = 1

dA_prev = np.full((m,1,1,1),0).astype(np.float64)
dA_prev_pad = np.pad(dA_prev,((0,0),(opadH,opadH),(opadW,opadW),(0,0)),mode="constant",constant_values=(0,0))
_,n_H_prev,n_W_prev,n_C_prev = dA_prev_pad.shape

W = np.full((2,2,1,number_of_filters),1).astype(np.float64)
fH,fW,n_C_prev,n_C = W.shape

n_H = int((n_H_prev-fH+2*opadH)/stride)+1
n_W = int((n_W_prev-fW+2*opadW)/stride)+1
dZ = np.full((m,n_H,n_W,n_C),1).astype(np.float64)



threadsperblock = (2,2,2)

blockspergrid_H = int(math.ceil(n_H_prev/threadsperblock[0]))
blockspergrid_W = int(math.ceil(n_W_prev/threadsperblock[1]))
blockspergrid_C = int(math.ceil(n_C_prev/threadsperblock[2]))

blockspergrid = (blockspergrid_H,blockspergrid_W,blockspergrid_C)

dA_prev_pad_device = cuda.to_device(dA_prev_pad)
dZ_device = cuda.to_device(dZ)
W_device = cuda.to_device(W)
cuda.synchronize()
    
func3D_new[blockspergrid,threadsperblock](dA_prev_pad_device,W_device,dZ_device,stride,n_H_prev,n_W_prev,n_C_prev)
cuda.synchronize()
    
    
dA_prev_pad = dA_prev_pad_device.copy_to_host()   
dA_prev = dA_prev_pad[:,opadH:-opadH,opadW:-opadW,:] 

print(dA_prev_pad.shape)
print(dA_prev)

print(dA_prev_pad)

(1, 3, 3, 1)
[[[[4.]]]]
[[[[4.]
   [4.]
   [4.]]

  [[4.]
   [4.]
   [4.]]

  [[4.]
   [4.]
   [4.]]]]


In [11]:
@cuda.jit
def add(dA_prev,Hlim,Wlim,Clim):

    IMG_H = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    IMG_W = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
    IMG_C = cuda.threadIdx.z + cuda.blockDim.z * cuda.blockIdx.z

    if IMG_H<Hlim and IMG_W<Wlim and IMG_C<Clim:

      m = dA_prev.shape[0]

      for i in range(m):

        dA_prev[i,IMG_H,IMG_W,IMG_C] = dA_prev[i,IMG_H,IMG_W,IMG_C] + 10


In [12]:
dA_prev = np.full((2,2,2,2),0)

m,n_H_prev,n_W_prev,n_C_prev = dA_prev.shape

threadsperblock = (2,2,2)

blockspergrid_H = int(math.ceil(n_H_prev/threadsperblock[0]))
blockspergrid_W = int(math.ceil(n_W_prev/threadsperblock[1]))
blockspergrid_C = int(math.ceil(n_C_prev/threadsperblock[2]))

blockspergrid = (blockspergrid_H,blockspergrid_W,blockspergrid_C)

dA_prev_device = cuda.to_device(dA_prev)
cuda.synchronize()

add[blockspergrid,threadsperblock](dA_prev_device,n_H_prev,n_W_prev,n_C_prev)

dA_prev = dA_prev_device.copy_to_host()
print(dA_prev)

[[[[10 10]
   [10 10]]

  [[10 10]
   [10 10]]]


 [[[10 10]
   [10 10]]

  [[10 10]
   [10 10]]]]


In [13]:
nH = 5
nW = 5

array = np.arange(25).reshape(5,5)
sum = np.zeros_like(array)

#get n_h tail
n_h_tail = nH - 1

for n_h in range(nH):

  if n_h > n_h_tail:

    break

  #get n_w tail
  n_w_tail = nW - 1

  for n_w in range(nW):
    
    if n_w > n_w_tail:

      break

    if (n_w == n_w_tail) and (n_h_tail == n_h):

      sum[n_h,n_w] = sum[n_h,n_w] + array[n_h,n_w]

    else:

      #calculate head 
      sum[n_h,n_w] = sum[n_h,n_w] + array[n_h,n_w]

      #calculate tail
      sum[n_h_tail,n_w_tail] = sum[n_h_tail,n_w_tail] + array[n_h_tail,n_w_tail]
      
    n_w_tail = n_w_tail - 1

  n_h_tail = n_h_tail - 1

print(array)
print("\n")
print(sum)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


[[ 0  1  2  0  0]
 [ 5  6  7  0  0]
 [10 11 12 13 14]
 [ 0  0 17 18 19]
 [ 0  0 22 23 24]]


In [14]:
nH = 5
nW = 5

array = np.arange(25).reshape(5,5)
sum = np.zeros_like(array)


for n_h in range(nH):

  #get n_w tail
  n_w_tail = nW - 1

  for n_w in range(nW):
    
    if n_w > n_w_tail:

      break

    if (n_w == n_w_tail):

      sum[n_h,n_w] = sum[n_h,n_w] + array[n_h,n_w]

    else:

      #calculate head 
      sum[n_h,n_w] = sum[n_h,n_w] + array[n_h,n_w]

      #calculate tail
      sum[n_h,n_w_tail] = sum[n_h,n_w_tail] + array[n_h,n_w_tail]
      
    n_w_tail = n_w_tail - 1


print(array)
print("\n")
print(sum)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


In [55]:
nH = 3
nW = 5

#array = np.arange(25).reshape(5,5)
#array = np.arange(12).reshape(3,4)
array = np.arange(15).reshape(3,5)
sum = np.zeros_like(array)

n_h_tail = nH - 1

for n_h in range(nH):

  if n_h > n_h_tail:

    break

  #get n_w tail
  n_w_tail = nW - 1

  for n_w in range(nW):
    
    if n_w > n_w_tail:

      break

    if (n_w == n_w_tail) or (n_h == n_h_tail): 

        if (n_h == n_h_tail) and (n_w == n_w_tail):

          sum[n_h,n_w] = sum[n_h,n_w] + array[n_h,n_w]#sum[n_h,n_w] = sum[n_h,n_w]/2

        else:

          sum[n_h,n_w] = sum[n_h,n_w] + array[n_h,n_w]
          sum[n_h_tail,n_w_tail] = sum[n_h_tail,n_w_tail] + array[n_h_tail,n_w_tail]

    else:

      #calculate head 
      sum[n_h,n_w] = sum[n_h,n_w] + array[n_h,n_w]

      #calculate tail
      sum[n_h_tail,n_w_tail] = sum[n_h_tail,n_w_tail] + array[n_h_tail,n_w_tail]

      #calculate top left hand corner
      sum[n_h_tail,n_w] = sum[n_h_tail,n_w] + array[n_h_tail,n_w]

      #calculate botton left hand corner
      sum[n_h,n_w_tail] = sum[n_h,n_w_tail] + array[n_h,n_w_tail]
      
    n_w_tail = n_w_tail - 1

  n_h_tail = n_h_tail -1

print(array)
print("\n")
print(sum)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
